In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1A8J'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1A8J.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex
the anti-idiotype 3H6 Fab (chains DC) only interact with the heavy chain (chain B) of 2F5 Fab. In AACDB, 2F5 Fab will be split and the light chain (chain A) will be discarded.

In [37]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1A8J
outputs dir:  ../outputs/1A/1A8J
[{'chains': [{'AA residues': 217,
              'chain_id': 'L',
              'seq': 'SALTQPPSASGSLGQSVTISCTGTSSDVGGYNYVSWYQQHAGKAPKVIIYEVNKRPSGVPDRFSGSKSGNTASLTVSGLQAEDEADYYCSSYEGSDNFVFGTGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVEKTVAPTECS'},
             {'AA residues': 217,
              'chain_id': 'H',
              'seq': 'SALTQPPSASGSLGQSVTISCTGTSSDVGGYNYVSWYQQHAGKAPKVIIYEVNKRPSGVPDRFSGSKSGNTASLTVSGLQAEDEADYYCSSYEGSDNFVFGTGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVEKTVAPTECS'}],
  'model_id': 0}]


In [38]:
print(p.structure.header['keywords'])
print(p.structure_id)

aspartame, ligand binding, immunotherapeutic agent, surrogate receptor, mcg(human) bence-jones dimer, immunoglobulin
1A8J


In [39]:
'''
anti-idiotype antibody binds to both the heavy and light chains of the antibody on the other side
heavy D + light C ~ receptor is heavy chain B
'''
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'immunoglobulin lambda light chain dimer (mcg)',
  'chain': 'l, h',
  'other_details': 'light chain dimer with aspartame'}}

In [40]:
# filter: is antibody-antigen complex
info = p.filter_antibody_antigen()
print(info)
p.update_chains('chain_type')

[{'L': 'light', 'H': 'light'}]
Pairwise chains of 1A8J are updated.


In [41]:
# split chains
p.chains_to_pdb()

Saved chain ['L'] to ../outputs/1A/1A8J/1A8J_L.pdb
Saved chain ['H'] to ../outputs/1A/1A8J/1A8J_H.pdb
Saved chain ['L', 'H'] to ../outputs/1A/1A8J/1A8J_L_H.pdb


In [42]:
p.bounded

[{'type': 'light-light',
  'chains': [['L'], ['H'], ['L', 'H']],
  'pdb_files': ['../outputs/1A/1A8J/1A8J_L.pdb',
   '../outputs/1A/1A8J/1A8J_H.pdb',
   '../outputs/1A/1A8J/1A8J_L_H.pdb']}]

## load SASA

heavy C + light B ~ receptor A
heavy E + light D ~ receptor A

In [11]:
# calculate SASA
sasa_file = p.build_freesasa()

In [12]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
light chain L ~ light chain H
------------------------------


In [13]:
# complex
p2.print_delta_sasa(0)

complex: light chain L ~ light chain H
------------------------------
light chain:
    chain_id res_no  res aa      value
36         L     38  TYR  Y  30.809926
38         L     40  GLN  Q  32.549124
40         L     42  ALA  A  20.514701
41         L     43  GLY  G  15.598749
42         L     44  LYS  K  26.860781
43         L     45  ALA  A  59.312465
44         L     46  PRO  P  79.530448
45         L     47  LYS  K   6.759702
46         L     48  VAL  V  41.528939
49         L     51  TYR  Y  59.307464
54         L     56  ARG  R   2.997984
55         L     57  PRO  P  39.206272
56         L     58  SER  S  28.217835
87         L     89  TYR  Y  44.384207
94         L     96  SER  S   4.082965
95         L     97  ASP  D  54.054281
96         L     98  ASN  N  37.491063
97         L     99  PHE  F  38.719451
99         L    101  PHE  F  79.714734
100        L    102  GLY  G   9.993281
101        L    103  THR  T   5.701865
116        L    118  THR  T   9.463583
118        L    120 

In [14]:
# retrieve sequences
motifs = p2.retrieve_seq()
motifs

([('light',
   [{'seq': 'YQQHAGKAPKVIIYEVNKRPS',
     'sig_res': 13,
     'start': (36, '38'),
     'end': (56, '58'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YCSSYEGSDNFVFGTGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSY',
     'sig_res': 38,
     'start': (87, '89'),
     'end': (179, '181'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'TECS',
     'sig_res': 3,
     'start': (211, '213'),
     'end': (214, '216'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'light',
   [{'seq': 'YVSWYQQHAGKAPKVIIYEVNKRPSGVPDRFSGSKSGNTASLTVSGLQAEDEADYYCSSYEGSDNFVFGTGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVEKTVAPTEC',
     'sig_res': 54,
     'start': (32, '34'),
     'end': (213, '215'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../outputs/1A/1A8J/seq_sasa_delta-0_span-30.p')

## distance

In [15]:
# calculate distance
dist_file = p.build_dist()
print(dist_file)

../outputs/1A/1A8J/dist.p


In [16]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()

key1 ['type', 'chains', 'pdb_files', 'dist']
light chain L ~ light chain H
------------------------------


In [17]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom     value aa
0         2  SER    C  8.472726  S
1         3  ALA    C  6.356459  A
2         4  LEU    C  7.986569  L
3         5  THR    C  9.660751  T
4         6  GLN    C  6.251604  Q
..      ...  ...  ...       ... ..
210     212  PRO    C  8.633336  P
211     213  THR    C  4.249182  T
212     214  GLU    C  5.060229  E
213     215  CYS    C  2.027732  C
214     216  SER    C  3.247496  S

[215 rows x 5 columns]
------------------------------
     res_no  res atom     value aa
32       34  TYR    C  4.117026  Y
36       38  TYR    C  3.203121  Y
38       40  GLN    C  2.959197  Q
42       44  LYS    C  3.727595  K
43       45  ALA    C  3.143795  A
44       46  PRO    C  3.424672  P
45       47  LYS    C  3.604265  K
46       48  VAL    C  3.600093  V
49       51  TYR    C  3.523182  Y
50       52  GLU    C  4.434381  E
55       57  PRO    C  4.163158  P
56       58  SER    C  3.910101  S
87       89  TYR    C  3.124050  Y
91  

In [18]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs[0]

[('light',
  [{'seq': 'YQQHAGKAPKVIIYEVNKRPS',
    'sig_res': 14,
    'start': (36, 38),
    'end': (56, 58),
    'dist_cutoff': 5,
    'max_span': 30},
   {'seq': 'YCSSYEGSDNFVFGTGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSY',
    'sig_res': 34,
    'start': (87, 89),
    'end': (179, 181),
    'dist_cutoff': 5,
    'max_span': 30},
   {'seq': 'TECS',
    'sig_res': 3,
    'start': (211, 213),
    'end': (214, 216),
    'dist_cutoff': 5,
    'max_span': 30}],
  'light',
  [{'seq': 'YVSWYQQHAGKAPKVIIYEVNKRPS',
    'sig_res': 12,
    'start': (32, 34),
    'end': (56, 58),
    'dist_cutoff': 5,
    'max_span': 30},
   {'seq': 'YCSSYEGSDNFVFGTGTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSY',
    'sig_res': 34,
    'start': (87, 89),
    'end': (179, 181),
    'dist_cutoff': 5,
    'max_span': 30},
   {'seq': 'EC',
    'sig_res': 2,
    'start': (212, 214),
    'end': (213, 215),
    'dist_cutoff': 5,
    'max_span': 30}])]